In [1]:
using Pkg
Pkg.add(;path="/Users/az/Documents/GitHub/GLOM_RV_Example")

   Updating git-repo `/Users/az/Documents/GitHub/GLOM_RV_Example`
   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
No Changes to `~/Documents/GitHub/GLOM_RV_Example/examples/Project.toml`
No Changes to `~/Documents/GitHub/GLOM_RV_Example/examples/Manifest.toml`


In [2]:

Pkg.activate("examples")
Pkg.instantiate()

 Activating environment at `~/Documents/GitHub/GLOM_RV_Example/examples/examples/Project.toml`


In [3]:
import Pkg; Pkg.add("CSV")

using Statistics

# for det()
using LinearAlgebra

# for importing the data from CSV
using DataFrames
using CSV


  Resolving package versions...
No Changes to `~/Documents/GitHub/GLOM_RV_Example/examples/examples/Project.toml`
No Changes to `~/Documents/GitHub/GLOM_RV_Example/examples/examples/Manifest.toml`


In [4]:
# For GLOM
# import Pkg; Pkg.add("GPLinearODEMaker")
import GPLinearODEMaker; GLOM = GPLinearODEMaker

GPLinearODEMaker

In [5]:
# For this module
using GLOM_RV_Example; GLOM_RV = GLOM_RV_Example

GLOM_RV_Example

In [6]:
# For units in orbit fitting functions
import Pkg; Pkg.add("Unitful")
using UnitfulAstro, Unitful

  Resolving package versions...
No Changes to `~/Documents/GitHub/GLOM_RV_Example/examples/examples/Project.toml`
No Changes to `~/Documents/GitHub/GLOM_RV_Example/examples/examples/Manifest.toml`


In [7]:

## Problem setup

# CHANGE: choose a kernel, I suggest 3 for Matern 5/2 or 4 for Quasi-periodic
# kernel
kernel_choice = 4
kernel_names = ["pp", "se", "m52", "qp", "m52_m52", "se_se"]
kernel_name = kernel_names[kernel_choice]
kernel_function, num_kernel_hyperparameters = GLOM.include_kernel(kernel_name)

(GPLinearODEMaker.qp_kernel, 3)

In [8]:
# CHANGE: the stars rotation rate which is used as the first guess for some GLOM
# hyperparameters and starting point for priors
star_rot_rate = 27.  # days for solar rotation
last = 100

# CHANGE: observation times go here
using DelimitedFiles
obs_xs = collect(Iterators.flatten(readdlm("./data/bjd_daily.txt")))[1:last]
# taking out the mean observation times makes the optimization easier for models
# with periodic parameters
GLOM_RV.remove_mean!(obs_xs)

100-element Array{Float64,1}:
 -94.0826835292828
 -93.08372810534638
 -92.08427081042464
 -91.08734937909321
 -90.08436831641302
 -89.08671614898776
 -88.08030606357352
 -87.1391856148548
 -85.07661038349761
 -75.07995068086893
 -74.09923327293654
 -73.1019451257016
 -72.05461601664138
   ⋮
  86.9183723042297
  90.89301300501393
  91.89041234012984
  96.91769183589349
 100.89068160001625
 101.88447341258143
 102.91082533841836
 103.8837681139994
 105.88832158272999
 106.91145564196631
 107.88312550192495
 108.88539447698713

In [9]:
# CHANGE: rvs and their errors go here
obs_rvs = collect(Iterators.flatten(readdlm("./data/rv_daily.txt")))[1:last]
# inject_ks = GLOM_RV.kep_signal(; K=50u"m/s", P=sqrt(2)*5u"d", M0=rand()*2*π, ω_or_k=rand()*2*π, e_or_h=0.1)
# obs_rvs[:] .+= ustrip.(inject_ks.(obs_xs.*u"d"))
obs_rvs_err = collect(Iterators.flatten(readdlm("./data/erv_daily.txt")))[1:last]

100-element Array{Float64,1}:
 0.03036812271298431
 0.028174782382301478
 0.02938168070678178
 0.030733648918260983
 0.030273630749804166
 0.02978809045216476
 0.03479668670704809
 0.05661399795951553
 0.049404085779396215
 0.02941258553740282
 0.07400877379839974
 0.05641940799521519
 0.05493576014838747
 ⋮
 0.14691235315354487
 0.05307828537878864
 0.042584554070920794
 0.06901435317592487
 0.05777880997525311
 0.05303811006842724
 0.05770075708886935
 0.04175327852466612
 0.04090507117229551
 0.05895118403438244
 0.040303889806123705
 0.03944750090751184

In [10]:
# CHANGE: activity indicators and thier errors go here
# you can actually have as many as you want, but obviously it will take longer
# to fit
obs_indicator1 = collect(Iterators.flatten(readdlm("./data/C1.txt")))[1:last]
obs_indicator1_err = collect(Iterators.flatten(readdlm("./data/err_C1.txt")))[1:last]
obs_indicator2 = collect(Iterators.flatten(readdlm("./data/C2.txt")))[1:last]
obs_indicator2_err = collect(Iterators.flatten(readdlm("./data/err_C2.txt")))[1:last]


100-element Array{Float64,1}:
  5.756971312004282
  5.3652157914954595
  5.654645807928711
  5.925730235674198
  5.825554595635701
  5.887501707941508
  6.627535360942655
 10.619782436423465
  9.48548099736605
  5.536845159896076
 13.636395158606499
 10.394849664742399
 10.086845919158254
  ⋮
 26.693044848802298
  9.892413073405644
  7.813029822015975
 12.731825673176225
 10.461033634526089
  9.842709673431468
 10.667208707539606
  7.744231382374497
  7.552010979571698
 10.58257459843627
  7.519403961091692
  7.233208313151117

In [11]:
# removing means as the GP model assumes zero mean
GLOM_RV.remove_mean!(obs_rvs)
GLOM_RV.remove_mean!(obs_indicator1)
GLOM_RV.remove_mean!(obs_indicator2)

100-element Array{Float64,1}:
  11.356363769515074
  18.369840882498668
  44.91653127212058
  32.12358939834826
  36.39076877769131
  26.73560618205582
  18.2190949910584
  13.588531603166306
  28.442060247312018
   8.399795401432204
  13.826736907087298
  16.945512031813177
  19.422727582445024
   ⋮
 -26.205247165443875
   0.5709376372474573
  -6.09410561647665
  24.37227136655505
  21.860624281480288
  -1.5398256067648468
  12.081312289070784
  14.362266371876192
   9.31883822574854
   3.3698728914795772
  -5.638480496250157
 -10.254162252573682

In [12]:
# CHANGE: change these lines if you add more than 2 indicators
# this takes the data and riffles it together so it takes the form
# [rv_1, ind1_1, ind2_1, rv_2, ind1_2, ind2_2, ...]
n_out = 3  # number of indicators + 1
obs_ys = collect(Iterators.flatten(zip(obs_rvs, obs_indicator1, obs_indicator2)))
obs_noise = collect(Iterators.flatten(zip(obs_rvs_err, obs_indicator1_err, obs_indicator2_err)))


300-element Array{Float64,1}:
  0.03036812271298431
  7.86437518965105
  5.756971312004282
  0.028174782382301478
  8.077654743316922
  5.3652157914954595
  0.02938168070678178
  7.700195206719579
  5.654645807928711
  0.030733648918260983
  7.8474301601074075
  5.925730235674198
  0.030273630749804166
  ⋮
  0.04090507117229551
 11.675368595519458
  7.552010979571698
  0.05895118403438244
 18.109730207587884
 10.58257459843627
  0.040303889806123705
 12.077619273006398
  7.519403961091692
  0.03944750090751184
 11.219388489133108
  7.233208313151117

In [13]:
# How many differention orders we want in the GLOM model
n_dif = 2 + 1

3

In [14]:
# CHANGE: consider changing a0 (the GLOM coefficients that are used, see
# commented lines below)
# If all a's active:
# problem_definition = GLOM.GLO(kernel_function, num_kernel_hyperparameters, n_dif, n_out, obs_xs, copy(obs_ys); noise=copy(obs_noise), a0=[[1. 1 1];[1 1 1];[1 1 1]])
problem_definition = GLOM.GLO(kernel_function, num_kernel_hyperparameters, n_dif, n_out, obs_xs, copy(obs_ys); noise=copy(obs_noise), a0=[[1. 1 0];[1 0 1];[1 0 1]])

# Makes the std of each output equal to 1, improves fitting stability
# the normalizations are stored in problem_definition.normals
GLOM.normalize_problem_definition!(problem_definition)

In [15]:
# CHANGE: Setting initial fit values
initial_total_hyperparameters = collect(Iterators.flatten(problem_definition.a0))
initial_hypers = [[star_rot_rate], [star_rot_rate], [star_rot_rate], [star_rot_rate, 2 * star_rot_rate, 1], [star_rot_rate, 2 * star_rot_rate, 1], [star_rot_rate, 2 * star_rot_rate, 1]]
append!(initial_total_hyperparameters, initial_hypers[kernel_choice])

12-element Array{Float64,1}:
  1.0
  1.0
  1.0
  1.0
  0.0
  0.0
  0.0
  1.0
  1.0
 27.0
 54.0
  1.0

In [16]:
## Fitting GLOM Model

# CHANGE: Setting kernel hyperparameter priors and kick function
# kick functions help avoid saddle points
tighten_lengthscale_priors = 3
if kernel_name in ["pp", "se", "m52"]
    kernel_hyper_priors(hps::Vector{<:Real}, d::Integer) =
        GLOM_RV.kernel_hyper_priors_1λ(hps, d, star_rot_rate, star_rot_rate / 2 / tighten_lengthscale_priors)
    add_kick!(hps::Vector{<:Real}) = GLOM_RV.add_kick_1λ!(hps)
elseif kernel_name == "qp"
    kernel_hyper_priors(hps::Vector{<:Real}, d::Integer) =
        GLOM_RV.kernel_hyper_priors_qp(hps, d, [star_rot_rate, 2 * star_rot_rate, 1], [star_rot_rate / 2, star_rot_rate / 2, 0.4] ./ tighten_lengthscale_priors)
    add_kick!(hps::Vector{<:Real}) = GLOM_RV.add_kick_qp!(hps)
elseif kernel_name in ["se_se", "m52_m52"]
    kernel_hyper_priors(hps::Vector{<:Real}, d::Integer) =
        GLOM_RV.kernel_hyper_priors_2λ(hps, d, [star_rot_rate, 2 * star_rot_rate, 1], [star_rot_rate / 2, star_rot_rate / 2, 1] ./ tighten_lengthscale_priors)
    add_kick!(hps::Vector{<:Real}) = GLOM_RV.add_kick_2λ!(hps)
else
    # kernel_hyper_priors(hps::Vector{<:Real}, d::Integer) = custom function
end




add_kick! (generic function with 1 method)

In [17]:
fit1_total_hyperparameters, result = GLOM_RV.fit_GLOM(problem_definition, initial_total_hyperparameters, kernel_hyper_priors, add_kick!)
# fit_GLOM returns a vector of num_kernel_hyperparameters gp hyperparameters
# followed by the GLOM coefficients and the Optim result object
workspace = GLOM.nlogL_matrix_workspace(problem_definition, fit1_total_hyperparameters)

gridsearching over hp7
starting at hp = 27.0 -> 10916.249644151492
searching from 20.25 to 33.75
ending at hp   = 27.0 -> 10916.249644151492
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 27.0, 54.0, 1.0]

[1.0600063708890808, 0.41177057200952627, 0.8657580034674567, 1.3830699880440993, 0.9670237537771093, 1.0042230929522638, 27.066116235728764, 53.97467352756599, 1.6924656233399067]

Iteration:              1
Time so far:            11.30447506904602 s
Unnormalized posterior: 5571.477268135242
Gradient norm:          4091.76671347133

[0.333625936193072, -0.7866636706661311, 0.3088864155373199, 2.3491732961292207, 0.3298805955898343, 0.9776382836810181, 27.66248195460705, 53.823798338124575, 2.1042783397022324]

Iteration:              2
Time so far:            22.52218198776245 s
Unnormalized posterior: 2630.8529490254914
Gradient norm:          3067.420675603089

[0.2814878838135825, -1.52934045894607, 0.9376153129348036, 2.3874306495512543, -1.7728463960556737, 0.9496501292487102, 25.4142385683847

GPLinearODEMaker.nlogL_matrix_workspace{Float64}([-0.5944904369357757, -0.5356196276683625, -0.9874265473936238, 2.469519904145497, 0.0, 0.0, 0.0, -0.5600321062369216, -0.08823700223611936, 25.29311926020973, 45.93456991339085, 6.582495576839502], Cholesky{Float64,Array{Float64,2}}([4.082049831826167 -0.1401134253612674 … 8.443774612183839e-5 -1.5087804921653467e-5; -0.5719499844325497 2.336936260032747 … -7.42235679172954e-5 -3.0295387435737433e-6; … ; 0.000344679087356431 -0.00018528660905384675 … 1.561284137262279 -0.06822650544238283; -6.158917154306154e-5 -4.9658349122760755e-6 … -0.00796937247974805 0.4510895793390472], 'U', 0), [-0.5944904369357757, -0.5356196276683625, -0.9874265473936238, 2.469519904145497, 0.0, 0.0, 0.0, -0.5600321062369216, -0.08823700223611936, 25.29311926020973, 45.93456991339085, 6.582495576839502], [[0.017357449703071157 -0.1154415344493876 … -0.001229579953298212 -0.006016134320549627; -0.07364027576166782 0.006489897428326595 … -0.0005980381196121092 0

In [18]:
## Plotting initial results

plot_xs = collect(LinRange(obs_xs[1]-10, obs_xs[end]+10, 300))
post, post_err, post_obs = GLOM_RV.GLOM_posteriors(problem_definition, plot_xs, fit1_total_hyperparameters)
GLOM_rvs_at_plot_xs, GLOM_ind1_at_plot_xs, GLOM_ind2_at_plot_xs = post
GLOM_rvs_err_at_plot_xs, GLOM_ind1_err_at_plot_xs, GLOM_ind2_err_at_plot_xs = post_err
GLOM_rvs_at_obs_xs, GLOM_ind1_at_obs_xs, GLOM_ind2_at_obs_xs = post_obs

Array dimensions do not match. The matrix can't be symmetric


3-element Array{Array{Float64,1},1}:
 [1.6675314966576853, 2.2116219434191495, 2.5218980617119047, 3.1229423937677527, 3.9177814263208424, 2.3804314363641903, 1.8169067978706677, 1.7759355868044218, 2.2420147485693445, -0.7810777253476199  …  -0.7413713411565512, 0.4657268845555395, 0.14897872984467347, 0.9974450441303505, 1.1539551206165746, 0.15784621796793577, -2.200970286425864, -4.180767957017474, -2.6348881445632126, -1.583756379439785]
 [-8.166083049118043, 10.68980470891454, -38.70707653208082, -12.916494667572008, -17.983433578266165, -33.52075636563325, -65.2110737392445, -17.15359634226396, -104.94266656788328, -34.03787714492622  …  20.93597700288253, 2.8144837147557125, 70.42869151119942, 38.81608104118394, 42.89887377431334, 36.57383749324839, 3.088316758396351, 44.16338504202108, 35.18163596195165, 45.8889406089299]
 [23.000431256098544, 27.260890132878565, 31.536192073729975, 30.817487659498944, 25.252978639646688, 20.82724564884524, 19.71943826060347, 15.8894329555274,

In [21]:
plot_xs = collect(LinRange(obs_xs[1]-10, obs_xs[end]+10, 300))
post, post_err, post_obs = GLOM_RV.GLOM_posteriors(problem_definition, plot_xs, fit1_total_hyperparameters)

Array dimensions do not match. The matrix can't be symmetric


3-element Array{Array{Array{Float64,1},1},1}:
 [[-3.416523674389593, 5.047644954489262, -2.2950276974438584, -1.169983936537997, 4.189655012766938, -1.389207860686583, 3.911659658717494, -0.9334152680777013, -2.62561763970614, -2.905498632211948  …  -3.0297307994236644, -1.6711443950166276, -0.23371175223172497, -1.145443349618886, -0.18269561011019406, 2.265507237392818, 0.6534606776089557, -4.350759298292009, 2.07987044218359, -1.9041991029577983], [-60.820328096116434, -32.679743914773155, 76.74721760396878, -20.12668139425965, 27.91060966730327, -48.93354413300465, 72.91366967484429, -17.967422176779852, -47.979101422312375, 11.64328375203502  …  9.267059787658127, -13.211228729641677, -33.941991569695304, 42.457453859911816, -54.03499038920713, -0.14120332656694964, 37.12954980128795, -33.71391114012009, 19.640982897342813, -37.71380121407705], [7.094169831057966, 7.2956443126028265, 5.575053331034287, 9.758932390410886, 7.698419609829167, 1.493738260459429, -0.7473854307359389, -

In [22]:
fit1_total_hyperparameters

12-element Array{Float64,1}:
 -0.5944904369357757
 -0.5356196276683625
 -0.9874265473936238
  2.469519904145497
  0.0
  0.0
  0.0
 -0.5600321062369216
 -0.08823700223611936
 25.29311926020973
 45.93456991339085
  6.582495576839502